#### 자연어 데이터셋 전처리<hr>
- Korpora 패키지 활용
- 데이터셋 : 한국어 혐오 데이터셋 korean_hate_speech

In [1]:
### 패키지 설치
# !pip install Korpora

In [2]:
### 패키지 설치 확인
import Korpora

Korpora.__version__

'0.2.0'

[1] 데이터 준비<hr>

In [3]:
### 데이터 다운로드
from Korpora import Korpora

# 데이터셋 파일명
data_file = 'korean_hate_speech'

# 다운로드 fetch
# corpus = Korpora.fetch(data_file)

In [4]:
### 데이터 로딩
corpus = Korpora.load(data_file)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [5]:
### 데이터 확인
hateDS = corpus.train # 학습용 데이터셋
hate_valDS = corpus.dev # 검증용 데이터셋
hate_testDS = corpus.test # 테스트용 데이터셋
unlabelDS = corpus.unlabeled # 프리딕트용 데이터셋

In [6]:
hateDS

KoreanHateSpeech.train: size=7896
  - KoreanHateSpeech.train.texts : list[str]
  - KoreanHateSpeech.train.titles : list[str]
  - KoreanHateSpeech.train.gender_biases : list[str]
  - KoreanHateSpeech.train.biases : list[str]
  - KoreanHateSpeech.train.hates : list[str]

In [7]:
### 데이터 확인
hateDS = corpus.train # 학습용 데이터셋
hate_valDS = corpus.dev # 검증용 데이터셋
hate_testDS = corpus.test # 테스트용 데이터셋
unlabelDS = corpus.unlabeled # 프리딕트용 데이터셋

(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....
"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"
False
others
hate


[2] 데이터 전처리 <hr>
- 주제에 따른 데이터 셋 선택 => 뉴스댓글 + 차별종류
- 형태소 분석기 결정

In [8]:
### 피쳐 데이터 추출
textData = corpus.get_all_texts()

In [9]:
### 데이터 파일 저장
save_filename = '../data/hate_text.csv'
with open(save_filename, mode = 'w', encoding='utf-8') as f:
    for text in textData:
        f.write(text + '\n')        


In [11]:
# !pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/991.7 kB ? eta -:--:--
   - -------------------------------------- 41.0/991.7 kB 1.9 MB/s eta 0:00:01
   ------------------ --------------------- 450.6/991.7 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------  983.0/991.7 kB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 991.7/991.7 kB 7.0 MB/s eta 0:00:00


In [17]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()

tokenizer.load('korean_hate_speech.model')

sentence = '안녕하세요. 저는 자연어 처리를 학습하고 있는 학생입니다.'
sentences = ['안녕하세요. 저는 자연어 처리를 학습하고 있는 학생입니다.', '이렇게 토크나이저가 잘 되는지 확인해보겠습니다.']

# 토크나이징
tokenized = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = tokenizer.encode_as_pieces(sentences)
print(f'[단일 문장 토큰화] : {tokenized}')
print(f'[여러 문장 토큰화] : {tokenized_sentences}')

# 인코딩
encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = tokenizer.encode_as_ids(sentences)
print(f'[단일 문장 정수 인코딩 ] : {encoded_sentence}')
print(f'[여러 문장 정수 인코딩 ] : {encoded_sentences}')

# 문장 변환
decode_ids = tokenizer.decode_ids(encoded_sentences)
decode_pieces = tokenizer.decode_pieces(encoded_sentence)
print(f'[정수 인코딩에서 문장 변환] : {decode_ids}')
print(f'[하위 단어 토큰에서 문장 변환] {decode_pieces}')


[단일 문장 토큰화] : ['▁안녕', '하세요', '.', '▁저는', '▁자연', '어', '▁처리', '를', '▁학', '습', '하고', '▁있는', '▁학생', '입니다', '.']
[여러 문장 토큰화] : [['▁안녕', '하세요', '.', '▁저는', '▁자연', '어', '▁처리', '를', '▁학', '습', '하고', '▁있는', '▁학생', '입니다', '.'], ['▁이렇게', '▁토크', '나이', '저', '가', '▁잘', '▁되는', '지', '▁확인', '해', '보', '겠습니다', '.']]
[단일 문장 정수 인코딩 ] : [4598, 1073, 30679, 27291, 4090, 30737, 11333, 31124, 816, 31105, 329, 1537, 14796, 4935, 30679]
[여러 문장 정수 인코딩 ] : [[4598, 1073, 30679, 27291, 4090, 30737, 11333, 31124, 816, 31105, 329, 1537, 14796, 4935, 30679], [1646, 3614, 2850, 30901, 30711, 290, 4116, 30690, 597, 30712, 30765, 3682, 30679]]
[정수 인코딩에서 문장 변환] : ['안녕하세요. 저는 자연어 처리를 학습하고 있는 학생입니다.', '이렇게 토크나이저가 잘 되는지 확인해보겠습니다.']
[하위 단어 토큰에서 문장 변환] 안녕하세요. 저는 자연어 처리를 학습하고 있는 학생입니다.
